# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [4]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

Título de la Web:
Frogames
Contenido de la Web:
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprenda

['#main-content',
 '/',
 '/',
 '/users/sign_in',
 'https://cursos.frogamesformacion.com/pages/rutas',
 'https://cursos.frogamesformacion.com/pages/instructores',
 'https://cursos.frogamesformacion.com/pages/certificaciones',
 'https://cursos.frogamesformacion.com/collections',
 '/courses/geometria-analitica-desde-cero',
 '/courses/geometria-analitica-desde-cero',
 '/courses/agentes-ia',
 '/courses/agentes-ia',
 '/courses/domina-android-desde-cero-kotlin-compose',
 '/courses/domina-android-desde-cero-kotlin-compose',
 '/courses/unity-ui',
 '/courses/unity-ui',
 '/courses/matematicas-ml-3',
 '/courses/matematicas-ml-3',
 '/courses/flutter-firebase-ai',
 '/courses/flutter-firebase-ai',
 '/courses/google-classroom',
 '/courses/google-classroom',
 '/courses/ia-productividad',
 '/courses/ia-productividad',
 '/courses/bandas-sonoras-4',
 '/courses/bandas-sonoras-4',
 '/courses/musica-videojuegos',
 '/courses/musica-videojuegos',
 '/courses/bandas-sonoras-3',
 '/courses/bandas-sonoras-3',
 '/c

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [5]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [6]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/geometria-analitica-desde-cero
/courses/geometria-analitica-desde-cero
/courses/agentes-ia
/courses/agentes-ia
/courses/domina-android-desde-cero-kotlin-compose
/courses/domina-android-desde-cero-kotlin-compose
/courses/unity-ui
/courses/unity-ui
/courses/matematicas-ml-3
/courses/matematicas-ml-3
/courses/flutter-firebase-ai
/courses/flutter-firebase-ai
/courses/google-classroom


In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-futures',
 'https://www.anthropic.com/transparency',
 'https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy',
 'http://trust.anthropic.com/',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/engineering',
 'https://docs.claude.com',
 'https://www.anthropic.com/company',
 'https://www.anthropic.com/careers',
 'https://www.anthropic.com/events',
 'https://www.anthropic.com/news',
 'https://claude.ai',
 'https://claude.com/product/overview',
 'https://claude.com/product/claude-code',
 'https://claude.com/platform/api',
 'https://claude.com/pricing',
 'https://claude.com/contact-sales',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://claude.ai/login',
 'https://console.anthropic.com',
 '#',
 'https://claude.ai/login',
 'https://cla

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'Pagina Sobre nosotros',
   'url': 'https://www.anthropic.com/'},
  {'type': 'Pagina de la empresa', 'url': 'https://www.anthropic.com/company'},
  {'type': 'Carreras/empleos disponibles',
   'url': 'https://www.anthropic.com/careers'},
  {'type': 'Pagina de Cursos', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'Pagina de Investigación',
   'url': 'https://www.anthropic.com/research'},
  {'type': 'Noticias', 'url': 'https://www.anthropic.com/news'},
  {'type': 'Transparencia', 'url': 'https://www.anthropic.com/transparency'},
  {'type': 'Eventos', 'url': 'https://www.anthropic.com/events'},
  {'type': 'Economía y futuros',
   'url': 'https://www.anthropic.com/economic-futures'},
  {'type': 'Confianza', 'url': 'http://trust.anthropic.com/'}]}

In [12]:
get_links("https://cursos.frogamesformacion.com")

{'links': [{'type': 'Pagina Inicio',
   'url': 'https://cursos.frogamesformacion.com'},
  {'type': 'Pagina Nuestros clientes',
   'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'},
  {'type': 'Pagina Frogames para empresas',
   'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'},
  {'type': 'Pagina Instructores',
   'url': 'https://cursos.frogamesformacion.com/pages/instructores'},
  {'type': 'Pagina Certificaciones',
   'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'},
  {'type': 'Pagina Premios',
   'url': 'https://cursos.frogamesformacion.com/pages/premios'},
  {'type': 'Pagina Afiliados',
   'url': 'https://cursos.frogamesformacion.com/pages/afiliados'},
  {'type': 'Opiniones en Trustpilot',
   'url': 'https://es.trustpilot.com/review/cursos.frogamesformacion.com'},
  {'type': 'Corporate Vision - Premios/educación y formación',
   'url': 'https://www.corporatevision-news.com/issues/education-and-training-awards-2023/8

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Pagina Inicio', 'url': 'https://www.anthropic.com/'}, {'type': 'Pagina Sobre nosotros', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Eventos', 'url': 'https://www.anthropic.com/events'}, {'type': 'Pagina de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}, {'type': 'Pagina de Transparencia', 'url': 'https://www.anthropic.com/transparency'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Skip to main content
Skip to footer
Research
Economic Futures
Commitments
Initiatives
Transparency
Responsible Scaling Policy
Trust center
Security and compliance
Learn
Learn
Anthropic Academy
Engineering at Anthropic
Developer docs
Company
About
Careers
Events
News
Try Claude
Try Claude
Try 

In [15]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [25]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}, {'type': 'Pagina de Eventos', 'url': 'https://www.anthropic.com/events'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Transparencia', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'Pagina de Futuro Económico', 'url': 'https://www.anthropic.com/economic-futures'}]}


'Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nSkip to main content\nSkip to footer\nResearch\nEconomic Futures\nCommitments\nInitiatives\nTransparency\nResponsible Scaling Policy\nTrust center\nSecurity and compliance\nLearn\nLearn\nAnthropic Academy\nEngineering at Anthropic\nDeveloper docs\nCompany\nAbout\nCareers\nEvents\nNews\nTry Claude\nTry Claude\nTry Claude\nLearn more about Claude\nProducts\nClaude\nClaude Code\nClaude Developer Platform\nPricing\nContact sales\nModels\nOpus\nSonnet\nHaiku\nLog in\nClaude.ai\nClaude Console\nEN\nThis is some text inside of a div block.\nLog in to Claude\nLog in to Claude\nLog in to Claude\nDownload app\nDownload app\nDownload app\nResearch\nEconomic Futures\nCommitments\nInitiatives\nTransparency\nResponsible Sca

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigacion', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Futuro economico', 'url': 'https://www.anthropic.com/economic-futures'}, {'type': 'Pagina de Transparencia', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'Pagina de Aprendizaje', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pagina de Eventos', 'url': 'https://www.anthropic.com/events'}, {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic: IA segura y responsable

Una empresa de IA centrada en la seguridad, la interpretabilidad y el control humano, dedicada a construir sistemas confiables para el beneficio de la humanidad.

## Quiénes somos
- Public Benefit Corporation con sede en San Francisco.
- Equipo interdisciplinario de investigadores, ingenieros, expertos en políticas y operadores.  
- Nuestro enfoque combina investigación de frontera, técnicas de seguridad y desarrollo de productos para mostrar cómo es posible una IA responsable.

## Nuestro enfoque y filosofía
- IA segura como ciencia: la seguridad de la IA se estudia y aplica de forma sistemática, buscando impactos positivos a largo plazo.
- Impacto humano: diseñamos herramientas que priorizan el beneficio humano y la confianza pública.
- Colaboración amplia: trabajamos con sociedad civil, gobiernos, academia y la industria para promover la seguridad en toda la industria.
- Interdisciplinariedad y apertura: compartimos aprendizajes para acelerar el progreso responsable.

## Productos y plataformas
- Claude: nuestra familia de modelos para uso general, con énfasis en seguridad y utilidad para empresas y usuarios.
- Claude Code: herramientas para desarrollo y codificación.
- Claude Developer Platform: plataforma para integrar y ampliar Claude en aplicaciones.
- Otras familias de modelos y soluciones como Opus, Sonnet y Haiku para distintos casos de uso.
- Enfoque orientado a agentes, codificación y tareas computacionales, con énfasis en manejo de contexto y control de comportamiento.

## Clientes e impacto
- Contamos con clientes, responsables de políticas y comunidades en la sociedad civil que se benefician de herramientas seguras y confiables.
- Trabajamos para que las soluciones basadas en Claude sirvan a negocios, organizaciones sin fines de lucro y grupos cívicos a nivel global.

## Cultura y valores
Nuestros principios guían cómo trabajamos y tomamos decisiones:
- Act for the global good: maximizar resultados positivos para la humanidad a largo plazo.
- Hold light and shade: entender y mitigar riesgos, junto con buscar beneficios.
- Be good to our users: ser generosos y atentos con usuarios, clientes y personas afectadas.
- Ignite a race to the top on safety: fomentar una competencia por IA más segura y robusta.
- Do the simple thing that works: soluciones empíricas y efectivas, priorizando el impacto real.
- Be helpful, honest, and harmless: confianza, integridad y reparación de relaciones cuando es necesario.
- Put the mission first: responsabilidad compartida y ownership para alcanzar la misión.

## Carreras en Anthropic
- Somos una prioridad para quienes quieren trabajar en un equipo diverso y colaborativo, con experiencia que va desde la física y ML hasta política pública y negocio.
- Puestos abiertos e entrevistas técnicas variadas (buscamos perspectivas distintas para avanzar en nuestra misión).
- Beneficios para empleados:
  - Salud, dental y visión para ti y tus dependientes.
  - Beneficios de fertilidad y 22 semanas de licencia parental.
  - Días libres flexibles y políticas de ausencia.
  - Apoyo mental y bienestar para ti y tu familia.
  - Paquetes competitivos de salario y equity, con opciones de donación de equity.
  - Planes de jubilación y protección de ingresos.
  - EstipENDios y ayudas: stipendio mensual para bienestar, transporte, educación anual, oficina en casa, reubicación y comidas en la oficina.
- Ambiente de trabajo: enfoque de alto rendimiento, bajo ego y alto nivel de confianza; misión compartida y propiedad individual para hacer que las cosas sucedan.

## Formación y desarrollo
- Anthropic Academy: aprende a construir con Claude.
- Cursos y recursos para desarrollar habilidades en IA segura, desarrollo de software y políticas públicas relacionadas con IA.
- Programas y materiales para futuros empleos dentro de Anthropic y en la industria de IA responsable.

## Gobernanza y ética
- Anthropic es una Public Benefit Corporation, orientada al desarrollo responsable y al mantenimiento de IA avanzada para el beneficio a largo plazo de la humanidad.
- Junta directiva y LTBT (Long-Term Benefit Trust) gestionan la gobernanza y la supervisión.
- Miembros de la Junta: Dario Amodei, Daniela Amodei, Yasmin Razavi, Jay Kreps y Reed Hastings.
- LTBT Trustees: Neil Buddy Shah, Kanika Bahl, Zach Robinson y Richard Fontaine.

## ¿Cómo unirse o saber más?
- Ver roles abiertos y aplicar: ver oportunidades de carrera en nuestra página de empleos.
- Explora Claude y la plataforma Claude Developer Platform para entender cómo trabajamos y qué ofrecemos a clientes y socios.
- Más información sobre nuestras políticas de seguridad, transparencia y ética en los apartados de Responsible Scaling Policy y Trust Center.

## ¿Te gustaría saber más?
- Explora Claude y las plataformas asociadas.
- Únete a nuestro equipo para construir IA más segura y confiable.
- Si eres cliente o socio, contacta para conocer cómo nuestras soluciones pueden ayudar a tus objetivos.

Footer breve
- © 2024 Anthropic PBC
- Productos: Claude, Claude Code, Claude Developer Platform
- Modelos: Opus, Sonnet, Haiku
- Servicios y políticas: Responsible Scaling Policy, Security and compliance, Privacy y Cookies policies

¿Quieres que adapte este folleto a un formato específico (una página, un tríptico, o una versión para inversores) o que lo adapte a un público concreto (clientes, inversores o nuevos empleados)?

In [20]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Pagina Nuestros clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Pagina Frogames para empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Pagina de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Pagina de Inicio de Cursos', 'url': 'https://cursos.frogamesformacion.com'}, {'type': 'Ruta de aprendizaje', 'url': 'https://cursos.frogamesformacion.com/pages/rutas'}, {'type': 'Instructores', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Reseñas de clientes', 'url': 'https://es.trustpilot.com/review/cursos.frogamesformacion.com'}, {'type': 'Curso: Geometría analítica desde cero', 'url': 'https://cursos.frogamesformacion.com/courses/geometria-analitica-desde-cero'}, {'type': 'Curso: Agentes IA', 'url': 'h

# Frogames Formación — Folleto breve

Una plataforma de aprendizaje online líder en educación digital y tecnológica, con más de 500,000 estudiantes y reconocimiento internacional.

---

## Quiénes somos
- Ganadores de premios internacionales por enseñanza online de datos y competencias digitales (Europa, 2023).
- Fundada por Juan Gabriel Gomila y María Santos, con un equipo docente de más de 20 instructores online.
- Somos una comunidad global de aprendizaje: aprender haciendo, con apoyo de expertos y una cultura orientada al progreso continuo.

- Premios y distinciones destacados:
  - Education and Training Awards 2023: Educación online más innovadora de datos y habilidades digitales (Europa)
  - The Silicon Review: Uno de los 30 líderes más inspiradores para seguir en 2023
  - CIO Bulletin: Una de las 50 empresas más innovadoras para seguir en 2023
  - Certificado de Excelencia 2023

---

## Qué hacemos

- Cursos online y formación de calidad para toda la familia.
- Temas clave: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
- Rutas temáticas: aprendizaje organizado paso a paso, subiendo de nivel con cada módulo.
- Instructores Expertos: soporte de un equipo de profesionales a lo largo del aprendizaje.
- Certificados blockchain: títulos verificados por blockchain para compartir en redes y mejorar tu CV.
- Actualizaciones constantes: nuevos cursos y actualizaciones semanales.
- Suscripciones y planes para acceder a todo el catálogo.

- Enfoque práctico y divertido: “Aprende mientras te diviertes”.

---

## Cultura y valores

- Comunidad de aprendizaje global y colaborativa.
- Enfoque práctico con casos reales y contenido avanzado.
- Soporte continuo de instructores y un entorno que fomenta la curiosidad y el crecimiento.
- Compromiso con la innovación educativa y la actualización constante de contenidos.

---

## Nuestros clientes y casos de éxito

- Más de 500,000 estudiantes satisfechos en distintos niveles y áreas.
- Colaboraciones con empresas y entidades a través de un catálogo de formaciones y rutas.
- Ejemplos de impacto en empresas: clientes que ven mejoras en productividad, contratación y gestión de talento gracias a la formación.

- Testimonios de estudiantes:
  - “Me encanta Frogames: cursos especializados y muy prácticos.” 
  - “La comunidad te ayuda a crecer y disfrutar aprendiendo.”
  - “El equipo humano es increíble: profesional y generoso con herramientas y apoyo.”

- Frogames para Empresas: programas orientados a formar a equipos de 5 o más personas, con planes escalables y certificados en la plataforma.

---

## Carreras y oportunidades

- Oportunidad de colaborar como afiliado: remuneración por cada venta que puedas generar si tienes una audiencia o grupo interesado en formación.
- Equipo docente establecido: más de 20 instructores online.
- Posibles vías de desarrollo profesional en el ámbito de la educación tecnológica y la formación online (dentro de Frogames Formación y su red de colaboradores).

- Enfoque de aprendizaje continuo para futuros empleados: rutas y cursos que puedes aprovechar para avanzar en tu carrera en tecnología, datos e IA.

---

## Cursos y packs para tu desarrollo

- Rutas de aprendizaje: 
  - Matemáticas desde Cero (Geometría Analítica, Aritmética)
  - Lenguajes de Programación (C#, Java, Kotlin, Python, Solidity, etc.)
  - Inteligencia Artificial, Machine Learning
  - Desarrollo Android/iOS
  - Desarrollo de Videojuegos (Unity, Unreal Engine)
  - Data & Analytics (Power BI, Tableau, SQL)
  - Blockchain, Trading Algorítmico
  - UX/UI, Marketing y otras habilidades
  - Otras áreas: Música para videojuegos, Teoría Musical, Ofimática, Bases de Datos, Desarrollo Web, etc.
- Certificados de finalización con verificación blockchain.
- Nuevos lanzamientos y actualizaciones semanales.
- Tipos de cursos destacados (ejemplos del catálogo):
  - Curso de Python de la A a la Z
  - Curso de Unreal Engine 5
  - Curso de IA para Android/Flutter
  - 15 días de Power BI – bootcamp completo
  - Unity 2021 – Desarrollo de videojuegos
  - Matemáticas para ML y fundamentos de matemáticas para ML

- Acceso mediante suscripción o compra de cursos individualmente.
- Entra gratis: curso introductorio de trading algorítmico con Python para conocer la plataforma, la comunidad y los certificados.

---

## Formación para Empresas

- Frogames para Empresas: ayuda a crecer a tus empleados y a tu negocio mediante un catálogo de cursos online y rutas temáticas, con descuentos para equipos.
- Ventajas: reducido coste de formación, actualizaciones continuas, certificados blockchain y contenido alineado con necesidades reales de negocio.
- Condiciones: planes para 5 o más trabajadores; precio sujeto a evaluación de necesidades; sin cursos personalizados con instructores en remoto o presencial.
- Propuesta a medida: solicita una propuesta sin compromiso indicando cantidad de empleados, periodo y consideraciones especiales.

- Resultados y valor para empresas: ahorro promedio de 80% en formación, ahorro de costes de contratación de más de 100,000€ al año, aumento de ingresos medios y productividad por empleado.

---

## Premios y confianza

- Frogames Formación y Juan Gabriel Gomila han sido reconocidos con múltiples premios y distinciones en educación digital y liderazgo en tecnología.

---

## Cómo empezar

- Prueba gratuita: accede a un curso gratuito para iniciar desde cero y descubrir nuestra plataforma, la comunidad, los certificados y las rutas.
- Solicita una propuesta para Empresas: envía información básica de tu empresa para recibir una propuesta personalizada.
- Solicita tu curso gratis y explora la plataforma hoy mismo.

---

Si quieres saber más o empezar ya, contáctanos o solicita una propuesta sin compromiso. ¡Bienvenido a Frogames Formación, donde aprender es un viaje que transforma tu vida profesional!

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Ofertas de empleo', 'url': 'https://www.anthropic.com/jobs'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Aprendizaje / Learn', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}, {'type': 'Pagina de Eventos', 'url': 'https://www.anthropic.com/events'}, {'type': 'Pagina de Transparencia', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'Pagina de Futuras Economía (Economic Futures)', 'url': 'https://www.anthropic.com/economic-futures'}, {'type': 'Pagina de Contacto de Ventas', 'url': 'https://www.anthropic.com/contact-sales'}, {'type': 'Investigación: Tracing Thoughts Language Model', 'url': 'https://www.anthropic.com/research/tracing-thoughts-language-model'}, {'

# Anthropic: IA segura para el beneficio a largo plazo

Anthropic es una empresa de investigación y seguridad en IA dedicada a construir sistemas confiables, interpretables y dirigibles. Somos una Public Benefit Corporation con sede en San Francisco y trabajamos para maximizar los beneficios de la IA al tiempo que mitigamos sus riesgos.

---

## Quiénes somos y qué hacemos

- Visión: La IA tendrá un impacto vasto en el mundo. Nos enfocamos en desarrollar sistemas en los que la gente pueda confiar y en generar investigación sobre oportunidades y riesgos.
- Enfoque: Seguridad como ciencia. Investigamos, aplicamos técnicas de seguridad en productos y compartimos aprendizajes para impulsar a toda la industria hacia IA más segura.
- Enfoques clave:
  - Investigación de frontera en IA
  - Política y responsabilidad
  - Productos prácticos que aprovechan la investigación (p. ej., Claude y herramientas afines)
- Productos y plataformas:
  - Claude (incluye Claude Sonnet, Haiku, Opus, y Claude Code)
  - Claude Developer Platform y Claude Console
  - Opciones de pricing y soluciones para diferentes necesidades (Team, Enterprise, etc.)
  - Recursos para desarrolladores y documentación

Ejecutamos, de forma continua, un marco de seguridad responsable como guía para el desarrollo de IA avanzada.

---

## Nuestra cultura y valores

En Anthropic trabajamos como un equipo interdisciplinario (investigadores, ingenieros, expertos en políticas, negocio y operaciones) con un objetivo común: IA confiable y útil para la sociedad. Nuestros valores guían cada decisión y acción:

1. Actuar para el bien global.
2. Mantener sombra y luz: entender riesgos y beneficios.
3. Ser buenos con nuestros usuarios (clientes, policy-makers y comunidades afectadas).
4. Impulsar una carrera hacia lo alto en seguridad.
5. Hacer lo sencillo que funciona.
6. Ser útiles, honestos e inofensivos.
7. Poner la misión en primer lugar.

Somos una empresa orientada a la misión, con un enfoque en seguridad, y fomentamos un entorno de confianza y colaboración para avanzar juntos.

---

## Carreras y cultura de trabajo

- Ubicación y equipo: Public Benefit Corporation con sede en San Francisco; equipos diversos con experiencia en física, ML, políticas públicas, negocios y más.
- Proceso de entrevistas: Diseñado para identificar candidatos con perspectivas diversas y fortalezas únicas; valoramos aportes de múltiples disciplinas.
- ¿Qué ofrecemos? Beneficios integrales para apoyar tu bienestar y desarrollo:
  - Salud, dental y visión para ti y tus dependientes
  - Beneficios de fertilidad a través de Carrot Fertility
  - 22 semanas de licencia parental pagada
  - Políticas flexibles de tiempo libre y ausencia
  - Apoyo en salud mental para ti y tus dependientes
  - Salario competitivo y paquetes de equity
  - Planes de retiro y opciones de salary sacrifice
  - Protección de vida e ingresos
  - Beneficios adicionales: estipendios de bienestar y ahorro de tiempo, beneficios para transporte, estipendio anual de educación, ayudas para oficina en casa, apoyo a la relocalización, comidas diarias en la oficina, entre otros
- Enfoque de equipo: Valoramos la colaboración entre investigación, producto y operaciones; todos contribuyen y cada rol es importante para lograr la misión.

---

## Anthropic Academy y aprendizaje continuo

- Anthropic Academy: Aprende a construir con Claude. Acceso a formación y recursos para desarrollar habilidades en IA responsable.
- Cursos y recursos: Presencia de cursos y materiales dentro de la oferta de aprendizaje (Learn) para ampliar conocimientos y habilidades relacionadas con Claude y plataformas afines.
- Orientación a futuros empleados: Enfoque práctico para preparar a futuros profesionales para trabajar con IA segura y herramientas de Anthropic.

---

## Gobernanza y compromiso público

- Modelo de negocio: Public Benefit Corporation, con el propósito de desarrollo y mantenimiento responsable de IA avanzada para el beneficio a largo plazo de la humanidad.
- Gobierno corporativo: Junta directiva con miembros como Dario Amodei, Daniela Amodei, Yasmin Razavi, Jay Kreps y Reed Hastings.
- LTBT (Fideicomiso de Beneficio a Largo Plazo): Personas designadas como trustees para garantizar el compromiso con el beneficio público.

---

## ¿Cómo participar o ponerse en contacto?

- ¿Buscas una carrera? Visita nuestras oportunidades laborales y descubre roles abiertos. Join us / See open roles.
- ¿Interesado en Claude u otros productos? Habla con ventas o prueba Claude. Try Claude / Speak with sales.
- ¿Quieres aprender más? Explora Anthropic Academy, la documentación para desarrolladores y las distintas iniciativas dentro de Learn y Seguridad y cumplimiento.

---

## Resumen para posibles clientes, inversores y futuros empleados

- Para clientes: Claude y su ecosistema ofrecen soluciones responsables y seguras para impulsar tus operaciones, con un marco de seguridad y políticas claras.
- Para inversores: Compromiso con la seguridad y la responsabilidad en la IA, con una estructura de gobernanza y un marco de escalado responsable (Responsible Scaling Policy).
- Para futuros empleados: Un entorno de trabajo basado en valores, con beneficios amplios, oportunidades de crecimiento y una sólida cultura de innovación y seguridad.

Si quieres explorar más, puedes:
- Probar Claude y conocer sus capacidades
- Consultar las oportunidades de empleo abiertas
- Acceder a Anthropic Academy para capacitación en Claude y desarrollo de IA responsable

Equipo, misión y productos alineados para construir un futuro de IA confiable y beneficioso para la sociedad.

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://huggingface.co/brand'}, {'type': 'Pagina de la Empresa', 'url': 'https://huggingface.co/enterprise'}, {'type': 'Carreras / Empleos', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Pagina de Cursos', 'url': 'https://huggingface.co/learn'}]}


# Folleto corporativo: Hugging Face

Un vistazo breve a la empresa que está dando forma al futuro de la IA abierta y colaborativa.

## ¿Quiénes somos?
- Hugging Face es la plataforma de la comunidad de ML que facilita la colaboración en modelos, datasets y aplicaciones.
- Misión: construir un futuro de IA abierto y ético, impulsando a ingenieros, científicos y usuarios finales a aprender, colaborar y compartir su trabajo.
- Somos la “comunidad de IA” y la casa del desarrollo de ML, con un enfoque fuerte en código abierto, educación y soporte a proyectos reales.

## Lo que ofrecemos
- Un hub central para compartir, explorar y experimentar con ML de código abierto.
- Acceso a un ecosistema con herramientas líderes como Transformers, Diffusers, Tokenizers, Datasets, PEFT y más.
- Multimodalidad: texto, imagen, video, audio e incluso 3D.
- Opciones de despliegue y escalabilidad para empresas y equipos: compute optimizado, endpoints de inferencia, y capacidades para Spaces.
- Comunidad global que impulsa la innovación y la ética en IA.

## Productos y plataformas clave
- Models, Datasets y Spaces: explora, aloja y crea aplicaciones de ML.
- Inference Endpoints: desplegar modelos de forma rápida y eficiente.
- Spaces: apps de ML creadas por la comunidad.
- Open Source Stack: Transformers, Diffusers, Safetensors, Hub Python Library, Tokenizers, TRL, Transformers.js, smolagents, PEFT, Accelerate, y más.
- Hugging Face Hub: el lugar central para compartir y descubrir modelos y datasets abiertos.
- Comunidad y aprendizaje: documentación, foros y recursos para todos los niveles.

## Clientes y casos de uso
- Más de 50,000 organizaciones ya confían en Hugging Face para sus proyectos de ML.
- Ejemplos de clientes y socios importantes: Google, Microsoft, NVIDIA, OpenAI, IBM Granite, Shopify, NVIDIA, Uber, Meta (AI at Meta) y otros grandes actores de la industria.
- Ofrecemos soluciones para equipos y empresas con seguridad, control de acceso y soporte dedicado, ideal para entornos corporativos.

## Cultura y valores
- Enfoque comunitario: la colaboración es la base de todo lo que hacemos.
- Innovación abierta: impulsamos una IA avanzada gracias a herramientas y modelos de código abierto.
- Compromiso con la ética y la apertura: construir un futuro de IA responsable y accesible para todos.
- En Hugging Face encontrarás un entorno de aprendizaje continuo, compartir conocimientos y apoyo mutuo entre desarrolladores, científicos de datos y usuarios.

## Carreras y empleos
- Páginas de empleo actuales (Current Openings) disponibles para conocer vacantes y roles.
- Oportunidad de unirse a un equipo que trabaja en un stack de código abierto de referencia y en soluciones empresariales de alto impacto.
- Si te apasiona la IA, la colaboración y la construcción de herramientas que empoderen a la comunidad, podrías encajar en Hugging Face.

## Cursos y programas de aprendizaje (Learn)
La sección Learn ofrece una variedad de cursos para ampliar habilidades en el ecosistema HF:
- LLM Course: introduce modelos grandes de lenguaje usando bibliotecas del ecosistema HF.
- MCP Course: Model Context Protocol.
- a smol course: el curso más corto sobre modelos post-entrenados.
- Agents Course: construir y desplegar tus propios agentes de IA.
- Deep RL Course: aprendizaje por refuerzo profundo con herramientas HF.
- Community Computer Vision Course: visión por computadora con herramientas HF.
- Audio Course: aplicar transformers a datos de audio.
- Open-Source AI Cookbook: notebooks de IA de código abierto para builders.
- ML for Games Course: incorporar IA en tus juegos y flujo de desarrollo.
- Diffusion Course: fundamentos de diffusion y su uso con diffusers.
- ML for 3D Course: ML aplicado a 3D con bibliotecas HF.

Además, hay otros recursos y cursos para ampliar tus habilidades en IA y ML dentro del ecosistema abierto.

## ¿Qué esperar en términos de precios y planes?
- Compute para equipos y proyectos: opciones de deployment y optimización, con precios desde $0.60 por hora para GPU.
- Planes para equipos y empresas: desde $20 por usuario/mes, con características como SSO, auditoría, gestión de recursos, seguridad avanzada y soporte prioritario.
- Opciones de Inference Providers, gestión de facturación y políticas de acceso para organizaciones.
- Ofrecemos una vía clara para escalar desde proyectos individuales hasta implementación corporativa con seguridad y cumplimiento.

## ¿Cómo empezar?
- Explora 1M+ modelos, datasets y aplicaciones en la plataforma.
- Aprovecha el stack de código abierto para mover tus proyectos más rápido.
- Construye tu portafolio ML para mostrar tu trabajo al mundo.
- Si eres empresa: evalúa las soluciones Enterprise o Team para una adopción con seguridad y soporte.

## Contacto y siguientes pasos
- Para clientes y equipos: consulta de precios, pruebas y despliegue de endpoints e infra para tus modelos.
- Para interesados en empleo: revisa las vacantes actuales y únete a un equipo que lidera la innovación en ML.
- Para inversionistas: Hugging Face ofrece una plataforma con una comunidad global, crecimiento de adopción y un ecosistema de herramientas de código abierto con uso real en grandes empresas.

Si quieres, puedo adaptar este folleto a un formato imprimible o a una versión enfocada para inversores, clientes o candidatos a empleo. ¿Prefieres algún formato específico o incluir un código QR/URL para contactar o explorar más?

In [24]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Pagina de Cursos - Rutas de aprendizaje', 'url': 'https://cursos.frogamesformacion.com/pages/rutas'}, {'type': 'Pagina de Instructores', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Pagina de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Pagina Nuestros Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Pagina Frogames para Empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Pagina de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Pagina Principal de Cursos', 'url': 'https://cursos.frogamesformacion.com'}, {'type': 'Facebook - Frogames', 'url': 'https://www.facebook.com/froggames/'}, {'type': 'Twitter - Frogames', 'url': 'https://twitter.com/frogames_sl'}, {'type': 'YouTube - Frogames Formación', 'url': 'https://www.youtube.com/channel/UCMUxXN

# Frogames Formación — Folleto breve

Una empresa líder en formación online en tecnología, inteligencia artificial y desarrollo digital para clientes, inversores y nuevos empleados.

## Quiénes somos
- Ganadores del premio: En 2023, reconocida como la opción de enseñanza online de datos y competencias digitales más innovadora de Europa.
- Misión: ayudarte a convertirte en experto en áreas como Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
- Enfoque: aprender divirtiéndose con cursos online de alta calidad para toda la familia.
- Más de 500,000 estudiantes satisfechos en todo el mundo de habla hispana.

## Nuestra oferta educativa
- Cursos online y rutas temáticas para aprender paso a paso.
- Temas clave: Matemáticas, Programación, IA, Desarrollo de Videojuegos, Data Science, Android, iOS, Unity, Unreal Engine, Blockchain, entre otros.
- Certificaciones verificados por blockchain para cada habilidad aprendida y lista para compartir en redes y CV.
- Actualizaciones semanales: nuevos cursos y mejoras continuas en las rutas.

### Rutas de aprendizaje (ejemplos)
- Matemáticas desde Cero, Matemáticas Avanzadas
- Inteligencia Artificial, Análisis de Datos, Business Intelligence
- Desarrollo de Videojuegos, Desarrollo Multiplataforma, Desarrollo Web
- Android, iOS, Blockchain, Trading Algorítmico
- Música y Composición, Ofimática y Productividad

### Cursos destacados (ejemplos)
- Aprende Matemáticas desde Cero — Geometría Analítica
- Curso Completo de Ingeniería de Agentes de IA (2025)
- Domina Android desde cero con Kotlin, Compose y Gemini AI
- Curso Completo de Python de la A a la Z
- Curso Completo de Unreal Engine 5
- Curso de Desarrollo de Videojuegos con Unity

## Nuestra cultura y comunidad
- Cultura de aprendizaje y crecimiento continuo: equipo de instructores expertos que acompaña a lo largo del proceso.
- Comunidad activa: más de 500k estudiantes y una comunidad de aprendizaje que comparte, colabora y progresa.
- Enfoque práctico y aplicado: casos reales, entornos de desarrollo y herramientas de última generación.
- Reconocimientos y premios en el sector educativo global.

## Instructores y talento
- Equipo de más de 20 instructores online, con experiencia en matemáticas, programación, IA y desarrollo de videojuegos.
- Representantes y colaboradores destacados, incluyendo figuras de referencia en educación y tecnología.

## Certificaciones y credenciales
- Certificaciones digitales validadas por blockchain para cada logro: curso, temática, nivel o ruta completada.
- Insignias y certificaciones visibles en LinkedIn para mejorar tu currículum.
- Portal de certificados disponible para gestionar y compartir fácilmente tus logros.

## Para empresas
- Frogames para Empresas: una vía para formar equipos con programas adaptados y una ventaja competitiva.
- Casos de éxito y formación de equipos en Power BI, desarrollo de software y más.
- Programa de afiliados para quien tenga audiencia o gestione grupos interesados en formación (remuneración por ventas).

## Carreras y empleo en Frogames
- Oportunidad de integrarte a un equipo docente y a una comunidad educativa con foco en tecnología.
- Más de 20 instructores y colaboradores en el ecosistema Frogames Formación.
- Cultura de aprendizaje, innovación y impacto global en la educación tecnológica.

## Cursos/packs para futuros empleos
- Rutas de aprendizaje con packs integrados: “Rutas de Aprendizaje” que aglutinan cursos fundamentales y avanzados en una única suscripción o pago único.
- Suscripciones con acceso a amplitud de contenido:
  - Rana de Bronce: acceso a 219 productos didácticos (IA, videojuegos, matemáticas, blockchain, etc.) por una cuota mensual.
  - Rana de Plata: opción de 6 meses.
  - Rana de Oro: suscripción anual.
  - Rana Black: paquete premium de alto acceso.
- Descuentos y opciones de actualizar contenido conforme evoluciona la tecnología.

## Muy pronto puedes empezar
- Acceso a un curso gratuito para iniciar en el mundo del trading algorítmico con Python.
- Exploración de rutas, certificados, y herramientas para promover tu progreso profesional.

## ¿Quieres saber más?
- Visita: Frogames Formación y explora las rutas, cursos y certificaciones.
- Si buscas colaborar o formar parte de nuestro equipo o programa de afiliados, te damos la bienvenida.

¿Listo para empezar? ¡Da el primer paso y descubre todo lo que Frogames Formación tiene para ti!

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.